In [12]:
# imports
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

In [13]:
# Read the Mapbox API key
pn.extension()
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token (mapbox_token)

# Import Data

In [14]:
# Import the CSVs to Pandas DataFrames
gold_csv_path = Path("Resources/gold_price.csv")
wages_csv_path = Path("Resources/minimum_hourly_wage_worldwide.csv")
countries_file_path = Path("Resources/countries.csv")

gold_price_df = pd.read_csv(gold_csv_path, index_col = 'Date', parse_dates=True, infer_datetime_format=True)
minimum_wage_df = pd.read_csv(wages_csv_path, index_col = 'Country')
df_country_locations = pd.read_csv(countries_file_path)
gold_df = gold_price_df.copy()

- - -

## Panel Visualizations

In [15]:
# Define Panel Visualization Functions
def daily_gold_price():
    """Daily GOLD Price."""
    gold_price_df.drop(columns = ['Open', 'High', 'Low', 'Change %'])        
    gold_price_df['Price'] = gold_price_df['Price'].str.replace(',', '')
    gold_price_df['Price'] = gold_price_df['Price'].astype(float)  
    
    daily_gold_price = gold_price_df.hvplot(
        title = 'GOLD Price from 2000 to 2019', 
        xlabel = 'Years', 
        ylabel = 'Price in USD'
    )
    return daily_gold_price


def hourly_minimum_wage():
    """Hourly Minimum Wage."""
    minimum_wage_df.dropna(inplace=True)
    minimum_wage_df.reset_index(inplace=True)
    minimum_wage_df.drop(columns = ['Country'], inplace=True)
    minimum_wage_df.rename(columns = {'Unnamed: 0':'Year'}, inplace=True)
    minimum_wage_df.index = minimum_wage_df.Year
    minimum_wage_df.drop(columns=['Year'], inplace=True)
    minimum_wage_df.index = pd.to_datetime(minimum_wage_df.index)
    minimum_wage_df.index = minimum_wage_df.index.year
    
    hourly_minimum_wage = minimum_wage_df.hvplot(
        height = 300, 
        width = 600, 
        xlabel = 'Year', 
        ylabel = 'Hourly Minimum Wage in USD', 
        title = 'Hourly Minimum Wage from 2000 to 2019'
    )
    return hourly_minimum_wage


def buying_power():
    """Buying Power."""
    gold_price_df = pd.read_csv(gold_csv_path, index_col = 'Date', parse_dates=True, infer_datetime_format=True)
    gold_price_df.drop(columns=['Open','High','Low','Change %'])        
    gold_price_df['Price'] = gold_price_df['Price'].str.replace(',', '')
    gold_price_df['Price'] = gold_price_df['Price'].astype(float)  
    
    gold_price_year_df = gold_price_df.copy()
    gold_price_year_df.index = gold_price_df.index.year
    avg_gold_price_df = gold_price_year_df.groupby('Date').mean()
    
    minimum_wage_df = pd.read_csv(wages_csv_path, index_col = 'Country')
    minimum_wage_df.dropna(inplace=True)
    minimum_wage_df.reset_index(inplace=True)
    minimum_wage_df.drop(columns = ['Country'], inplace=True)
    minimum_wage_df.rename(columns = {'Unnamed: 0':'Year'}, inplace=True)
    minimum_wage_df.index = minimum_wage_df.Year
    minimum_wage_df.drop(columns=['Year'], inplace=True)
    minimum_wage_df.index = pd.to_datetime(minimum_wage_df.index)
    minimum_wage_df.index = minimum_wage_df.index.year
   
    minimum_wage_df['Germany'].replace(
        '..', minimum_wage_df
        .loc[2015:2019, 'Germany']
        .astype(float)
        .mean(), 
        inplace=True
    )
    minimum_wage_df['Japan'].replace(
        '..', minimum_wage_df
        .loc[2001:2018, 'Japan']
        .astype(float)
        .mean(), 
        inplace=True
    )  

    for column in minimum_wage_df.columns:
        minimum_wage_df[column] = minimum_wage_df[column].astype(float)
    work_hours = pd.concat([avg_gold_price_df, minimum_wage_df], axis=1)
    
    for Country in work_hours.columns:
        if Country != 'Price':
            work_hours[Country] = work_hours['Price'] / work_hours[Country]        
    work_hours.dropna(inplace=True)
        
    buying_power = work_hours.hvplot(
        title = 'Number of Hours to Purchase 1 oz of Gold', 
        xlabel = 'Year', 
        ylabel = 'Hours'
    )
    return buying_power


def MC_GOLD():
    """Buying Power."""  
    gold_df = gold_df[['Open', 'High', 'Low', 'Price', 'Change %']]
    gold_df.rename(
        columns = {'Open':'open', 
                 'High':'high', 
                 'Low':'low', 
                 'Price':'close', 
                 'Change %':'volume'},
        inplace=True
    )
    
    for columns in gold_df.columns:
        if columns != 'volume': 
            gold_df[columns] = gold_df[columns].str.replace(',', '')
            gold_df[columns] = gold_df[columns].astype(float)
        
    gold_df.columns = pd.MultiIndex.from_product([['GOLD'], gold_df.columns])
    
    from MCForecastTools import MCSimulation

    num_sims = 500

    MC_GOLD = MCSimulation(
        portfolio_data = gold_df,
        num_simulation = num_sims,
        weights = [1],
        num_trading_days = 252 * 5
    )

    MC_GOLD.calc_cumulative_return()
    line_plot = MC_GOLD.plot_simulation()
    tbl = MC_GOLD.summarize_cumulative_return()
    ci_lower = round(tbl[8]*1500,2)
    ci_upper = round(tbl[9]*1500,2)    
    return MC_GOLD


def hourly_minimum_wage_by_year():
    """Hourly minimum wage by year."""
    minimum_wage_df.dropna(inplace=True)
    minimum_wage_df.reset_index(inplace=True)
    minimum_wage_df.drop(columns = ['Country'], inplace=True)
    minimum_wage_df.rename(columns = {'Unnamed: 0':'Year'}, inplace=True)
    minimum_wage_df.index = minimum_wage_df.Year
    minimum_wage_df.drop(columns=['Year'], inplace=True)
    minimum_wage_df.index = pd.to_datetime(minimum_wage_df.index)
    minimum_wage_df.index = minimum_wage_df.index.year
   
    minimum_wage_df['Germany'].replace(
        '..', minimum_wage_df
        .loc[2015:2019, 'Germany']
        .astype(float)
        .mean(), 
        inplace=True
    )
    minimum_wage_df['Japan'].replace(
        '..', minimum_wage_df
        .loc[2001:2018, 'Japan']
        .astype(float)
        .mean(), 
        inplace=True
    )  

    for column in minimum_wage_df.columns:
        minimum_wage_df[column] = minimum_wage_df[column].astype(float)
        
    def hourly_minimum_wage(year): 
        s_title = 'Hourly Minimum Wage around the World for '+str(year);
        return minimum_wage_df.loc[year, :].hvplot.bar(height=350, width=700, title=s_title, xlabel='Country', ylabel='Hourly Minimum Wage in USD', rot=90)
            
    hourly_minimum_wage_by_year = interact(hourly_minimum_wage, year=minimum_wage_df.index.values.tolist())  
    return hourly_minimum_wage_by_year


def highest_minimum_wage():
    """Top 5 countries with highest minimum wage."""
    minimum_wage_df.dropna(inplace=True)
    minimum_wage_df.reset_index(inplace=True)
    minimum_wage_df.drop(columns = ['Country'], inplace=True)
    minimum_wage_df.rename(columns = {'Unnamed: 0':'Year'}, inplace=True)
    minimum_wage_df.index = minimum_wage_df.Year
    minimum_wage_df.drop(columns=['Year'], inplace=True)
    minimum_wage_df.index = pd.to_datetime(minimum_wage_df.index)
    minimum_wage_df.index = minimum_wage_df.index.year
   
    minimum_wage_df['Germany'].replace(
        '..', minimum_wage_df
        .loc[2015:2019, 'Germany']
        .astype(float)
        .mean(), 
        inplace=True
    )
    minimum_wage_df['Japan'].replace(
        '..', minimum_wage_df
        .loc[2001:2018, 'Japan']
        .astype(float)
        .mean(), 
        inplace=True
    )  
    
    highest_minimum_wage_parallel = minimum_wage_df[['Australia','Belgium','France','Luxembourg','Netherlands']]
    highest_minimum_wage_parallel.reset_index(inplace=True);
    
    highest_minimum_wage = px.parallel_coordinates(
        highest_minimum_wage_parallel, 
        color='Year'
    )
    return highest_minimum_wage

def buying_power_mapbox():
    """Number of Hours to Purchase 1 oz of Gold in 2019."""
    gold_price_df.drop(columns=['Open','High','Low','Change %'], inplace=True)        
    gold_price_df['Price'] = gold_price_df['Price'].str.replace(',', '')
    gold_price_df['Price'] = gold_price_df['Price'].astype(float)   
    gold_price_year_df = gold_price_df.copy()
    gold_price_year_df.index = gold_price_df.index.year
    avg_gold_price_df = gold_price_year_df.groupby('Date').mean()
    
    minimum_wage_df.dropna(inplace=True)
    minimum_wage_df.reset_index(inplace=True)
    minimum_wage_df.drop(columns = ['Country'], inplace=True)
    minimum_wage_df.rename(columns = {'Unnamed: 0':'Year'}, inplace=True)
    minimum_wage_df.index = minimum_wage_df.Year
    minimum_wage_df.drop(columns=['Year'], inplace=True)
    minimum_wage_df.index = pd.to_datetime(minimum_wage_df.index)
    minimum_wage_df.index = minimum_wage_df.index.year
   
    minimum_wage_df['Germany'].replace(
        '..', minimum_wage_df
        .loc[2015:2019, 'Germany']
        .astype(float)
        .mean(), 
        inplace=True
    )
    minimum_wage_df['Japan'].replace(
        '..', minimum_wage_df
        .loc[2001:2018, 'Japan']
        .astype(float)
        .mean(), 
        inplace=True
    )  
    
    work_hours = pd.concat([avg_gold_price_df, minimum_wage_df], axis=1)
    for Country in work_hours.columns:
        if Country != 'Price':
            work_hours[Country] = work_hours['Price'] / work_hours[Country]       
    
    work_hours.dropna(inplace=True)
    
    work_hours_mapbox = pd.DataFrame(work_hours.drop(columns=['Price']).loc[2019, :]).reset_index().rename(columns={'index':'Country', 2019:'work_hours'})
    work_hours_mapbox_merged = pd.merge(df_country_locations, work_hours_mapbox)
    s_title="Number of Hours to Purchase 1 oz of Gold in 2019";
    buying_power_mapbox = px.scatter_mapbox(work_hours_mapbox_merged, lat="Lat", lon="Long", color="Country", title=s_title, size="work_hours",
                        size_max=10, height=760, zoom=1, range_color=(100, 1200))
    buying_power_mapbox.update_layout(mapbox_style="light", mapbox_access_token=mapbox_token)
    return buying_power_mapbox


In [16]:
daily_gold_price()
hourly_minimum_wage()
buying_power()
MC_GOLD()
hourly_minimum_wage_by_year()
highest_minimum_wage()
buying_power_mapbox()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

## Panel Dashboard

In [ ]:
row1 = pn.Row(function(), function())
row2 = pn.Row(function(), function())
row3 = pn.Row(function(), function())
row4 = pn.Row(function(), function())

In [ ]:
column1 = pn.Column(
    "## Column description goes here", row1
    #daily_gold_price()
)
column2 = pn.Column(
    "## Column description goes here.",
    row1
)
column3 = pn.Column(
    "## Column description goes here.", 
    row2
)
column4 = pn.Column(
    "## Column description goes here.", 
    row3
)

tabs = pn.Tabs(
    ("Tab NAME", column1), 
    ("Tab NAME", column2), 
    ("Tab NAME", column3), 
    ("Tab NAME", column4)
)

## Serve the Panel Dashboard

In [ ]:
tabs.servable()